## What Is The Effect Of The Earth's Temperature on Cyclonic Storms?

- https://berkeleyearth.org/data/
- https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C01552
- https://www.spc.noaa.gov/wcm/#data

        - https://www.spc.noaa.gov/wcm/data/SPC_severe_database_description.pdf

- https://www.ncei.noaa.gov/sites/g/files/anmtlf171/files/2025-04/IBTrACS_version4r01_Technical_Details.pdf
- https://www.ncei.noaa.gov/sites/default/files/2021-07/IBTrACS_v04_column_documentation.pdf

> Basin - All storms that have at least one position in that basin. This allows analysis of a
given basin but also means that different basin files should not be combined since some
storms will be in both files.

>SID* A unique storm identifier (SID) assigned by IBTrACS algorithm

### Earth's Temperature

In [ ]:
import pandas as pd
import numpy as np
import re

NH_PATH = "NH.Ts+dSST.csv"
SH_PATH = "SH.Ts+dSST.csv"

MONTHS = ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
MONTH_NUM = {m:i+1 for i, m in enumerate(MONTHS)}

def read_and_long(path, hemisphere: str) -> pd.DataFrame:
    df = pd.read_csv(path, skip_blank_lines=True, header=1)
    # Standardize headers and drop auto Unnamed cols
    df.columns = [c.strip() for c in df.columns]
    df = df.loc[:, ~df.columns.str.match(r"^Unnamed")]

    # Find a year column, fallback to first column
    year_col = next((c for c in ["Year","year","YEAR","Date","date"] if c in df.columns), df.columns[0])

    # Map many header variants to Jan..Dec
    month_alias = {
        "JAN":"Jan","FEB":"Feb","MAR":"Mar","APR":"Apr","MAY":"May","JUN":"Jun",
        "JUL":"Jul","AUG":"Aug","SEP":"Sep","OCT":"Oct","NOV":"Nov","DEC":"Dec",
        "Jan":"Jan","Feb":"Feb","Mar":"Mar","Apr":"Apr","May":"May","Jun":"Jun",
        "Jul":"Jul","Aug":"Aug","Sep":"Sep","Oct":"Oct","Nov":"Nov","Dec":"Dec",
        "JAN.":"Jan","FEB.":"Feb","MAR.":"Mar","APR.":"Apr","MAY.":"May","JUN.":"Jun",
        "JUL.":"Jul","AUG.":"Aug","SEP.":"Sep","OCT.":"Oct","NOV.":"Nov","DEC.":"Dec",
    }

    rename = {}
    month_cols = []
    for c in df.columns:
        if c == year_col:
            continue
        token = re.sub(r"[^A-Za-z]", "", c)  # strip digits, dots, asterisks
        if token in month_alias:
            rename[c] = month_alias[token]
            month_cols.append(c)

    # If detection failed, fall back to literal Jan..Dec if present
    if not month_cols:
        for m in MONTHS:
            if m in df.columns:
                month_cols.append(m)
                rename[m] = m

    # Rename and keep only Year + months
    if rename:
        df = df.rename(columns=rename)
    keep = [year_col] + [m for m in MONTHS if m in df.columns]
    df = df[keep]

    # Numeric coercion
    df[year_col] = pd.to_numeric(df[year_col], errors="coerce")
    for m in keep[1:]:
        df[m] = pd.to_numeric(df[m], errors="coerce")

    # Long format
    long_df = df.melt(id_vars=[year_col], var_name="month", value_name="temp")
    long_df = long_df.dropna(subset=[year_col, "month"]).copy()
    long_df["hemisphere"] = hemisphere

    # Build a month midpoint date
    year_int = long_df[year_col].apply(lambda x: int(np.floor(x)) if pd.notna(x) else np.nan)
    long_df["date"] = pd.to_datetime(
        {"year": year_int, "month": long_df["month"].map(MONTH_NUM), "day": 15},
        errors="coerce"
    )

    long_df = long_df.rename(columns={year_col: "year"})
    long_df = long_df.sort_values(["hemisphere", "year", "month"], kind="mergesort").reset_index(drop=True)
    return long_df[["hemisphere","year","month","date","temp"]]

# Build outputs
nh_long = read_and_long(NH_PATH, "NH")
sh_long = read_and_long(SH_PATH, "SH")
temperature_df = pd.concat([nh_long, sh_long], ignore_index=True)

### Hurricanes & Tropical Cyclones

### Tornadoes